# Instalar pacotes e configurar ambiente

Baixa e instala pacotes necessários

In [ ]:
!pip install --upgrade google-colab pandas gspread duckdb

Importa as dependências

In [2]:
from google.colab import drive
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import duckdb

Conecta com o seu Google Drive

In [3]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Define a variáveis da pasta de saída do arquivo Parquet e cria no Google Drive

In [4]:
output_folder = '/content/drive/MyDrive/Sheets2DuckDB/Output/Parquet'
!mkdir -p '$output_folder'

Autoriza a conexão com o Google Sheets

In [5]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Importar planilha do Google Sheets e criar Dataframe do resultado

Ingere dados da planilha do Google Sheets através da URL

In [19]:
# Planilha de exemplo
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1eWKSLabBJCQ99a7xTG3_Kt6fi_V4YJuspv8PbYUSWYM'
spreadsheet = gc.open_by_url(spreadsheet_url)

Importa dados da primeira aba da planilha

In [20]:
worksheet = spreadsheet.get_worksheet(0)

Cria um DataFrame da planilha e obtém todos os dados página de index(0)

In [21]:
df = pd.DataFrame(worksheet.get_all_records())

# Limpar dados e realizar agregações

Limpa dados, define formato, cria novas colunas de ANO e MÊS a partir da DATA

In [22]:
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%Y', errors='coerce')
df['VALOR'] = df['VALOR'].str.replace('R\$', '', regex=True).str.replace(',', '.', regex=True).str.replace(' ', '', regex=True)
df['VALOR'] = df['VALOR'].astype(float)
df['ANO'] = df['DATA'].dt.year
df['ANO'].fillna(0, inplace=True)
df['ANO'] = df['ANO'].astype(int)
df['MES'] = df['DATA'].dt.month
df['MES'].fillna(0, inplace=True)
df['MES'] = df['MES'].astype(int)

Agrega por ANO, MÊS, TIPO e CATEGORIA, novo VALOR com a soma do resultado e apenas duas casas decimais, e cria a coluna ID como índice

In [23]:
df_total_ano_mes_por_tipo_categoria = df.groupby(['ANO', 'MES', 'TIPO', 'CATEGORIA'])['VALOR'].sum().round(2).reset_index()
df_total_ano_mes_por_tipo_categoria['ID'] = range(1, len(df_total_ano_mes_por_tipo_categoria) + 1)
df_total_ano_mes_por_tipo_categoria = df_total_ano_mes_por_tipo_categoria[['ID', 'ANO', 'MES', 'TIPO', 'CATEGORIA', 'VALOR']]

# Salvar Dataframes em arquivo Parquet na pasta de saída do Google Drive

Cria e salva o DataFrame em um arquivo parquet na pasta "output" definida na variável

In [24]:
df_total_ano_mes_por_tipo_categoria.to_parquet(f'{output_folder}/total_ano_mes_por_tipo_categoria.parquet')

# DuckDB

# Criar banco de dados e tabela a partir do Parquet

Cria um banco de dados e uma conexão, importa o arquivo Parquet e cria uma tabela

In [26]:
con = duckdb.connect('sheets2duckdb.db')
con.execute(f"CREATE TABLE IF NOT EXISTS total_entrada_por_categoria AS SELECT * FROM read_parquet('{output_folder}/total_ano_mes_por_tipo_categoria.parquet')")

Consulta a tabela criada

In [33]:
con.query("SELECT * FROM total_entrada_por_categoria")

┌───────┬───────┬───────┬─────────┬──────────────────┬─────────┐
│  ID   │  ANO  │  MES  │  TIPO   │    CATEGORIA     │  VALOR  │
│ int64 │ int64 │ int64 │ varchar │     varchar      │ double  │
├───────┼───────┼───────┼─────────┼──────────────────┼─────────┤
│     1 │  2023 │     5 │ ENTRADA │ MERCH            │   180.0 │
│     2 │  2023 │     5 │ ENTRADA │ TATUAGEM         │  4666.0 │
│     3 │  2023 │     5 │ SAIDA   │ INSUMOS ATELIE   │  203.98 │
│     4 │  2023 │     5 │ SAIDA   │ MARKETING        │  239.41 │
│     5 │  2023 │     5 │ SAIDA   │ MATERIAIS TATTOO │ 1384.72 │
│     6 │  2023 │     6 │ ENTRADA │ MERCH            │   500.0 │
│     7 │  2023 │     6 │ ENTRADA │ TATUAGEM         │ 3475.49 │
│     8 │  2023 │     6 │ SAIDA   │ INSUMOS ATELIE   │  2683.7 │
│     9 │  2023 │     6 │ SAIDA   │ MATERIAIS TATTOO │ 1049.43 │
│    10 │  2023 │     7 │ ENTRADA │ MERCH            │   240.0 │
│    11 │  2023 │     7 │ ENTRADA │ TATUAGEM         │ 2080.74 │
│    12 │  2023 │     7 │